<a href="https://colab.research.google.com/github/antbartash/australian_rain/blob/main/MicrosoftNNI/BOGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Microsoft NNI on Google Colab

In [1]:
 # install NNI and dependencies
! pip install nni==2.9.0
! pip install filelock==3.10
! pip install "typeguard<3"

 # download ngrok and unzip it
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.2.7
    Uninstalling hyperopt-0.2.7:
      Successfully uninstalled hyperopt-0.2.7
ERROR: pip's dependency resolver does not currently take into account a

In [2]:
! ./ngrok authtoken 2ZqSaEGuFWdJVe4YADpDuGtZ1R9_2L3tHZaAHSSavE6N96jo4 # YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# clone the github repo to get the model.py file
! git clone https://github.com/antbartash/australian_rain.git

Cloning into 'australian_rain'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 269 (delta 91), reused 105 (delta 56), pack-reused 101
Receiving objects: 100% (269/269), 8.75 MiB | 13.22 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [4]:
from nni.experiment import Experiment

In [5]:
hyperparameter_space = {
    'n_estimators': {'_type': 'randint', '_value': [100, 500]},
    'learning_rate': {'_type': 'loguniform', '_value': [1e-6, 0.5]},
    'depth': {'_type': 'randint', '_value': [1, 6]},
    'l2_leaf_reg': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'random_strength': {'_type': 'uniform', '_value': [0.0, 100.0]}, # CPU only
    'bagging_temperature': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'grow_policy': {'_type': 'choice', '_value': ['SymmetricTree', 'Depthwise']}, # GPTuner support only numerical values
    'scale_pos_weight': {'_type': 'uniform', '_value': [1, 4]}
}

In [6]:
experiment = Experiment('local')
experiment.config.experiment_name = 'catboost_nni_bogp'
experiment.config.tuner.name = 'GPTuner'
experiment.config.tuner.class_args = {
    'optimize_mode': 'maximize',
    'utility': 'ei', # aquisition function
    'xi': 0.0,
    'cold_start_num': 5
}
experiment.config.trial_command = 'python3 /content/australian_rain/MicrosoftNNI/model_GPU.py'
experiment.config.trial_code_directory = '.'
experiment.config.search_space = hyperparameter_space
experiment.config.max_trial_number = 10
experiment.config.trial_concurrency = 1
experiment.config.max_experiment_duration = '1h'

In [7]:
get_ipython().system_raw('./ngrok http 5000 &')

In [8]:
! curl -s http://localhost:4040/api/tunnels # don't change the port number 4040

In [9]:
experiment.run(5000, wait_completion=True, debug=False)

[2023-12-30 11:19:27] Creating experiment, Experiment ID: z3ontrcy


INFO:nni.experiment:Creating experiment, Experiment ID: ${CYAN}z3ontrcy


[2023-12-30 11:19:27] Starting web server...


INFO:nni.experiment:Starting web server...


[2023-12-30 11:19:28] Setting up...


...

[2023-12-30 11:20:17] WARNING: KeyboardInterrupt detected


[2023-12-30 11:20:17] Stopping experiment, please wait...


INFO:nni.experiment:Stopping experiment, please wait...


[2023-12-30 11:20:17] Experiment stopped


INFO:nni.experiment:Experiment stopped


In [ ]:
# ValueError: 'x0' must only have one dimension.